**Sprint 5: Model Improvement**
**Goal:** Improve model accuracy and optimize their parameters.

**Tasks:**

1. **Hyperparameter Optimization**

   * Use GridSearchCV or RandomizedSearchCV to find the best model parameters.

2. **Adding Additional Features**

   * Add new features, such as the average wind speed over the past 12 hours.

3. **Comparing Models with Enhanced Features**

   * Retrain all models with the new features and compare the results.

4. **Visualizing Improved Results**

   * Create an interactive graph showing the improvement in accuracy.


In [13]:
from tomodachi_core.models.weather_impact_model.tomodachi_model import TomodachiModel
from tomodachi_core.tomodachi.services import PandasService
from config_loader import load_config
import os
import pathlib
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.metrics import root_mean_squared_error, r2_score
from sklearn.compose import ColumnTransformer
import xgboost as xgb
import numpy as np
# get curr dir
current_dir = os.getcwd()

# find the root using patlihb
root_dir = pathlib.Path(current_dir).parents[0].resolve()

# path to the config
config_path = (root_dir / "tomodachi_core" / "config_development" / "config.py").resolve()

# load the config
config = load_config(config_path)

# Grab the CSV_PATH
CSV_PATH = config.SAVE_TO_PATH

# Finally, we have to combine the path
CSV_PATH = (root_dir / CSV_PATH).resolve()

# Create PandasService
pandas_service = PandasService(str(CSV_PATH))

# load the data
df = pandas_service.load_csv_data()

Path C:\Users\Lenovo\Desktop\python_app\tuuleenergia_tomodachi exists.


C:\Users\Lenovo\Desktop\python_app\tuuleenergia_tomodachi\tomodachi_core\tomodachi\internals\service.py:47: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df[col] = pd.to_datetime(df[col])


In [14]:
print(df.head(6))

            Timestamp  Wind_Speed  Wind_Gust  Wind_Direction  Temperature  \
0 2020-01-01 00:00:00    0.265077   4.876651             274     2.152885   
1 2020-01-01 01:00:00   10.727089  13.030088             232    -9.783598   
2 2020-01-01 02:00:00   16.280163  17.651853             175     2.125048   
3 2020-01-01 03:00:00    5.110434   9.387124             235    -0.174307   
4 2020-01-01 04:00:00   16.444265  20.349305               1     1.881483   
5 2020-01-01 05:00:00   17.641930  18.989849             352     8.668347   

    Humidity  Precipitation     Pressure  Cloud_Cover  Solar_Radiation  \
0  88.345267       8.519070   997.887007    18.112024       359.985040   
1  70.172549       0.604355  1021.258081    76.901148       414.169416   
2  73.305597       2.239670   983.310126    49.615723       150.986509   
3  60.924091       1.314233  1032.161577    48.975636       879.761958   
4  78.893660       9.442267   951.217727    48.506623       752.151006   
5  76.722646    

### Experiment - 2

What is going: we try to use Scaler but dont balance yet

In [3]:
from tomodachi_core.tomodachi.services.preprocess import PreprocessData

df["Power_Output"] = df["Power_Output"].shift(-1)
df["Wind_Speed_12h_avg"] = df["Wind_Speed"].rolling(window=12, min_periods=1).mean()
df["Wind_Gust_Diff"] = df["Wind_Gust"] - df["Wind_Speed"]
df["Radiation_Lag1"] = df["Solar_Radiation"].shift(1)

X = df.drop(columns=["Power_Output", "Timestamp"]) # Precipitation units are gone here
y = df["Power_Output"]

#X = X.dropna() # remove any NaN that ocurred during the SHIFT

"""
Coming FROM SPRINT-3: Weather stuff
"""

weather_cols = ['Temperature', 'Humidity', 'Wind_Speed', 'Wind_Gust', 'Pressure', 'Solar_Radiation']

time_features = X[["Hour_of_Day", "Day_of_Week", "Month"]]

for col in weather_cols:
    X[f'{col}_6h_avg'] = df[col].rolling(window=6).mean().shift(1)  # shift 1 hour to avoid leakage

X["Power_Output"] = y  # add target back in temporarily
X = X.dropna()
y = X["Power_Output"]
X = X.drop(columns=["Power_Output"])

split_index = int(len(X) * 0.8)

X_train = X.iloc[:split_index]
y_train = y.iloc[:split_index]

X_test = X.iloc[split_index:]
y_test = y.iloc[split_index:]

num_features = ['Temperature', 'Humidity', 'Wind_Speed', 'Wind_Gust', 'Pressure'] + [f'{c}_6h_avg' for c in weather_cols]
mini_max_feature = ['Solar_Radiation']
cat_features = ['Precipitation_Unit', 'Hour_of_Day', 'Day_of_Week', 'Month']  # if you want to treat time features categorically

preprocessor = PreprocessData().preprocess(X, y)

preprocessor.set_feature_groups(
    num_features=num_features,
    mini_max_feature=mini_max_feature,
    cat_features=cat_features
)

# Step 3: Fit on training data
X_train_processed = preprocessor.fit_transform(X_train)

# Step 4: Transform test data
X_test_processed = preprocessor.transform(X_test)

In [4]:
X

,Wind_Speed,Wind_Gust,Wind_Direction,Temperature,Humidity,Precipitation,Pressure,Cloud_Cover,Solar_Radiation,Hour_of_Day,...,Precipitation_Unit,Wind_Speed_12h_avg,Wind_Gust_Diff,Radiation_Lag1,Temperature_6h_avg,Humidity_6h_avg,Wind_Speed_6h_avg,Wind_Gust_6h_avg,Pressure_6h_avg,Solar_Radiation_6h_avg
6,4.483481,7.472077,135,3.559107,60.753164,2.975750,1029.558057,36.022696,72.049464,6,...,mm,10.136063,2.988596,945.304449,0.811643,74.727302,11.078160,14.047478,989.317487,583.726396
7,16.282654,20.290289,139,-9.475298,74.072225,2.662896,1020.248024,23.155608,623.508954,7,...,mm,10.904387,4.007636,72.049464,1.046013,70.128618,11.781227,14.480049,994.595995,535.737134
8,12.260810,11.707539,233,-1.116112,67.132593,4.553474,1000.500509,19.690190,596.184975,8,...,mm,11.055100,-0.553271,623.508954,1.097397,70.778564,12.707154,15.690083,994.427653,570.627057
9,10.557190,10.863124,48,6.604642,62.840877,2.146955,992.571269,61.535854,572.929661,9,...,mm,11.005309,0.305934,596.184975,0.557203,69.749730,12.037262,14.699364,997.292716,644.826801
10,12.196672,15.272509,250,-0.364625,72.021647,3.277789,1027.533048,20.504884,258.605936,10,...,mm,11.113615,3.075837,572.929661,1.687028,70.069194,12.945055,14.945364,990.694332,593.688085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29994,19.474157,24.335100,219,13.280240,62.507313,7.844448,1019.498918,99.591344,744.643055,18,...,mm,17.363944,4.860944,511.732782,18.069408,52.775266,16.224869,18.040771,990.984619,486.220799
29995,14.038603,15.448871,33,17.966169,58.951727,1.671178,974.805014,96.604976,327.931566,19,...,mm,16.606346,1.410268,744.643055,17.324622,55.270075,16.762905,19.107607,997.561883,601.429408
29996,12.260810,20.936323,113,20.851254,45.662581,5.376946,1010.168380,45.460144,124.431132,20,...,mm,16.704059,8.675513,327.931566,17.823118,54.667907,18.111178,19.886054,987.229036,492.250729
29997,7.267274,9.518739,294,26.768642,54.446902,4.640283,1030.399016,74.430948,363.282709,21,...,mm,15.619569,2.251465,124.431132,18.012431,53.449367,18.301013,21.355521,992.264597,467.210886


In [5]:
X_train_processed

array([[-0.39489085, -0.40085367, -0.96230775, ...,  0.        ,
         0.        ,  0.        ],
       [-1.66952363,  0.7894847 ,  0.55039583, ...,  0.        ,
         0.        ,  0.        ],
       [-0.85207988,  0.16928254,  0.03477857, ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.63957443,  1.6101168 ,  0.06063365, ...,  0.        ,
         0.        ,  0.        ],
       [-0.57668795, -1.333258  , -0.0084273 , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.06511482,  1.74019371, -0.51276496, ...,  0.        ,
         0.        ,  0.        ]], shape=(23994, 59))

In [6]:
# instantiate the model & fit
model = TomodachiModel(X_train_processed, y_train)
model.fit_models()

Fitting was successful!
R2 score: -0.1144
RMSE: 461.7425
Fitting was successful!


In [7]:
from sklearn.model_selection import train_test_split
print("Prediction for the testing: ", model.xgboost_model.predict(X_test_processed))

Prediction successful!
Prediction for the testing:  Ok([391.79877 328.51376 373.9062  ... 641.2318  631.3345  532.9025 ])


In [11]:
param_grid = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

In [12]:
grid_search_tomodachi = GridSearchCV(
    estimator=model.xgboost_model.model,
    param_grid=param_grid,
    scoring='r2',
    cv=3,
    verbose=1,
    n_jobs=-1,
)


grid_search_tomodachi.fit(X_train_processed, y_train)

best_model_tomodachi = grid_search_tomodachi.best_estimator_
print("Best parameters:", grid_search_tomodachi.best_params_)
print("Best R2 score on train:", grid_search_tomodachi.best_score_)

Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.01, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Best R2 score on train: -0.0008963461659231194


### Experiment - 1

This experiment attempts to see how Model behaves for pure data

In [11]:
# We use X and y here
# While I drop: Precipitation_Unit 
# I keep that in mind!
X_experiment_1 = df.drop(columns=["Power_Output", "Precipitation_Unit", "Timestamp"])
y_experiment_1 = df["Power_Output"]

In [12]:
X_experiment_1

,Wind_Speed,Wind_Gust,Wind_Direction,Temperature,Humidity,Precipitation,Pressure,Cloud_Cover,Solar_Radiation,Hour_of_Day,Day_of_Week,Month,Wind_Speed_Squared,Wind_Speed_Cubed
0,0.265077,4.876651,274,2.152885,88.345267,8.519070,997.887007,18.112024,359.985040,0,2,1,0.070266,0.018626
1,10.727089,13.030088,232,-9.783598,70.172549,0.604355,1021.258081,76.901148,414.169416,1,2,1,115.070448,1234.370982
2,16.280163,17.651853,175,2.125048,73.305597,2.239670,983.310126,49.615723,150.986509,2,2,1,265.043714,4314.954915
3,5.110434,9.387124,235,-0.174307,60.924091,1.314233,1032.161577,48.975636,879.761958,3,2,1,26.116535,133.466829
4,16.444265,20.349305,1,1.881483,78.893660,9.442267,951.217727,48.506623,752.151006,4,2,1,270.413853,4446.757073
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,14.038603,15.448871,33,17.966169,58.951727,1.671178,974.805014,96.604976,327.931566,19,5,6,197.082370,2766.761113
29996,12.260810,20.936323,113,20.851254,45.662581,5.376946,1010.168380,45.460144,124.431132,20,5,6,355.631369,6706.564301
29997,7.267274,9.518739,294,26.768642,54.446902,4.640283,1030.399016,74.430948,363.282709,21,5,6,52.813270,383.808500
29998,17.993004,20.718309,332,28.712751,42.605638,1.539080,993.555765,41.156055,791.169689,22,5,6,323.748190,5825.202451


In [13]:
# Split these guys
X_experiment_1_train, X_experiment_1_test, y_experiment_1_train, y_experiment_1_test = train_test_split(X_experiment_1, y_experiment_1)

In [14]:
X_experiment_1_train

,Wind_Speed,Wind_Gust,Wind_Direction,Temperature,Humidity,Precipitation,Pressure,Cloud_Cover,Solar_Radiation,Hour_of_Day,Day_of_Week,Month,Wind_Speed_Squared,Wind_Speed_Cubed
16796,4.554862,5.230213,272,12.336390,68.931663,4.340127,1003.788346,62.217558,533.918625,20,1,11,20.746768,94.498664
12517,6.284050,7.973354,54,17.631828,46.672095,5.808992,1044.603351,27.593604,502.927467,13,5,6,39.489281,248.152609
22399,5.537283,10.277236,291,6.604642,40.294212,2.541787,1041.848759,64.486798,15.127418,7,4,7,30.661503,169.781415
17286,3.305013,3.362661,160,-0.673933,83.073872,9.293353,997.276287,49.220565,500.488981,6,1,12,10.923111,36.101025
22411,22.892997,24.974384,170,12.623885,40.236849,9.350432,1037.647120,21.841835,368.203993,19,4,7,524.089300,11997.974649
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12420,21.146743,22.896900,139,16.031160,68.705847,5.587806,1040.631301,81.795617,919.578609,12,1,6,447.184754,9456.501230
2256,0.167946,5.095862,101,2.198588,77.069786,7.733759,977.994508,31.208246,902.778278,0,5,4,0.028206,0.004737
17120,6.020279,6.889350,139,-2.295523,89.859950,7.871563,994.853441,21.051231,596.173974,8,1,12,36.243764,218.197582
27928,10.421883,13.049575,125,12.210623,72.681499,6.855163,1007.547246,95.269004,79.912851,16,3,3,108.615642,1131.979499


In [70]:
# instantiate the model & fit
model_experiment_1 = TomodachiModel(X_experiment_1_train, y_experiment_1_train) # How do you think? Will this fail miserably lol? Noooooooooooooo 
# And ladies and gentlemen! This is exactly why our X must exclude that Precipiation units!!!!! YESSSS! NOOOO!
model_experiment_1.fit_models()

# And look at this! Freaking 0.9158! For no preprocessed data! LMAO! We are crazy good!

Fitting was successful!
R2 score: 0.9158
RMSE: 125.7540
Fitting was successful!


In [71]:
# No we gonna run. Wait. Yes. We gonna slap GridSearchCV here
param_grid_experiment_1 = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

grid_search_tomodachi_experiment_1 = GridSearchCV(
    estimator=model_experiment_1.xgboost_model.model,
    param_grid=param_grid_experiment_1,
    scoring='r2',
    cv=3,
    verbose=1,
    n_jobs=-1,
)


grid_search_tomodachi_experiment_1.fit(X_experiment_1_train, y_experiment_1_train)

best_model_tomodachi = grid_search_tomodachi_experiment_1.best_estimator_
print("Best parameters:", grid_search_tomodachi_experiment_1.best_params_)
print("Best R2 score on train:", grid_search_tomodachi_experiment_1.best_score_)

Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'subsample': 1.0}
Best R2 score on train: 0.9276159922665271


### Results

We see that R2 is already pretty good for no preprocessed data. However, we must ask ourselves. What kind of thing we predict here?

Model predicts Power_Output based on X features we chose. Some features might be redundant. We must figure it out later.

### Experiment 3

What is going on here? I try to use Semen features.

In [19]:
# Avoid modifying the current df
df_experiment_3 = df.copy()

# Define features
df_experiment_3['T-WG'] = df_experiment_3['Wind_Gust']*df_experiment_3['Temperature'] # Wind Gust x Temperature
df_experiment_3['H-WG'] = df_experiment_3['Wind_Gust']*df_experiment_3['Humidity'] # Wind Gust x Humidity
df_experiment_3['CC-WG'] = df_experiment_3['Wind_Gust']*df_experiment_3['Cloud_Cover'] # Wind Gust x Cloud Cover
df_experiment_3['SR-WG'] = df_experiment_3['Wind_Gust']*df_experiment_3['Solar_Radiation'] # Wind Gust x Solar Radiation
df_experiment_3['T-WS'] = df_experiment_3['Wind_Speed']*df_experiment_3['Temperature'] # Wind Speed x Temperature
df_experiment_3['H-WS'] = df_experiment_3['Wind_Speed']*df_experiment_3['Humidity'] # Wind Speed x Humidity
df_experiment_3['CC-WS'] = df_experiment_3['Wind_Speed']*df_experiment_3['Cloud_Cover'] # Wind Speed x Cloud Cover
df_experiment_3['SR-WS'] = df_experiment_3['Wind_Speed']*df_experiment_3['Solar_Radiation'] # Wind Speed x Solar Radiation

In [20]:
df_experiment_3

,Timestamp,Wind_Speed,Wind_Gust,Wind_Direction,Temperature,Humidity,Precipitation,Pressure,Cloud_Cover,Solar_Radiation,...,Power_Output,Precipitation_Unit,T-WG,H-WG,CC-WG,SR-WG,T-WS,H-WS,CC-WS,SR-WS
0,2020-01-01 00:00:00,0.265077,4.876651,274,2.152885,88.345267,8.519070,997.887007,18.112024,359.985040,...,6.001146,mm,10.498870,430.829065,88.326027,1755.521531,0.570681,23.418342,4.801090,95.423931
1,2020-01-01 01:00:00,10.727089,13.030088,232,-9.783598,70.172549,0.604355,1021.258081,76.901148,414.169416,...,128.699686,mm,-127.481143,914.354479,1002.028719,5396.663892,-104.949531,752.747207,824.925493,4442.832363
2,2020-01-01 02:00:00,16.280163,17.651853,175,2.125048,73.305597,2.239670,983.310126,49.615723,150.986509,...,370.222084,mm,37.511030,1293.979601,875.809441,2665.191624,34.596124,1193.427081,807.752073,2458.085011
3,2020-01-01 03:00:00,5.110434,9.387124,235,-0.174307,60.924091,1.314233,1032.161577,48.975636,879.761958,...,-30.746944,mm,-1.636239,571.901971,459.740342,8258.434161,-0.890783,311.348546,250.286752,4495.965393
4,2020-01-01 04:00:00,16.444265,20.349305,1,1.881483,78.893660,9.442267,951.217727,48.506623,752.151006,...,495.256687,mm,38.286876,1605.431155,987.076072,15305.750342,30.939609,1297.348250,797.655765,12368.570504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,2023-06-03 19:00:00,14.038603,15.448871,33,17.966169,58.951727,1.671178,974.805014,96.604976,327.931566,...,396.956699,mm,277.557030,910.737616,1492.437803,5066.172421,252.219916,827.599880,1356.198893,4603.701016
29996,2023-06-03 20:00:00,12.260810,20.936323,113,20.851254,45.662581,5.376946,1010.168380,45.460144,124.431132,...,653.633256,mm,436.548589,956.006539,951.768255,2605.130365,255.653266,559.860232,557.378192,1525.626479
29997,2023-06-03 21:00:00,7.267274,9.518739,294,26.768642,54.446902,4.640283,1030.399016,74.430948,363.282709,...,46.179854,mm,254.803732,518.265868,708.488799,3457.993444,194.535057,395.680547,540.910088,2640.074957
29998,2023-06-03 22:00:00,17.993004,20.718309,332,28.712751,42.605638,1.539080,993.555765,41.156055,791.169689,...,517.191273,mm,594.879651,882.716779,852.683877,16391.698266,516.628639,766.603408,740.521062,14235.519322


In [25]:
X_experiment_3 = df_experiment_3[['T-WG', 'H-WG', 'CC-WG',	'SR-WG', 'T-WS', 'H-WS', 'CC-WS', 'SR-WS']]
y_experiment_3 = df_experiment_3['Power_Output']

In [26]:
X_experiment_3

,T-WG,H-WG,CC-WG,SR-WG,T-WS,H-WS,CC-WS,SR-WS
0,10.498870,430.829065,88.326027,1755.521531,0.570681,23.418342,4.801090,95.423931
1,-127.481143,914.354479,1002.028719,5396.663892,-104.949531,752.747207,824.925493,4442.832363
2,37.511030,1293.979601,875.809441,2665.191624,34.596124,1193.427081,807.752073,2458.085011
3,-1.636239,571.901971,459.740342,8258.434161,-0.890783,311.348546,250.286752,4495.965393
4,38.286876,1605.431155,987.076072,15305.750342,30.939609,1297.348250,797.655765,12368.570504
...,...,...,...,...,...,...,...,...
29995,277.557030,910.737616,1492.437803,5066.172421,252.219916,827.599880,1356.198893,4603.701016
29996,436.548589,956.006539,951.768255,2605.130365,255.653266,559.860232,557.378192,1525.626479
29997,254.803732,518.265868,708.488799,3457.993444,194.535057,395.680547,540.910088,2640.074957
29998,594.879651,882.716779,852.683877,16391.698266,516.628639,766.603408,740.521062,14235.519322


In [27]:
# Split into train/test
X_experiment_3_train, X_experiment_3_test, y_experiment_3_train, y_experiment_3_test = train_test_split(X_experiment_3, y_experiment_3)

# See the model
model_experiment_3 = TomodachiModel(X_experiment_3_train, y_experiment_3_train)
model_experiment_3.fit_models()

"""
R2 score: 0.8019
RMSE: 195.5892
Fitting was successful!
"""

Fitting was successful!
R2 score: 0.8065
RMSE: 192.3344
Fitting was successful!


'\nR2 score: 0.8019\nRMSE: 195.5892\nFitting was successful!\n'

In [ ]:
# TODO: Update params and use voting regressor, tune xgboost

param_grid_experiment_3 = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

param_grid_experiment_3_xgb = {
    'xgb__colsample_bytree': [0.8, 1.0],
    'xgb__learning_rate': [0.01, 0.05, 0.1],
    'xgb__n_estimators': [100, 200, 300],
    'xgb__max_depth': [3, 5, 7],
    'xgb__subsample': [0.8, 1.0]
}

param_grid_experiment_3_forest = {
    'memory': [None, '/tmp'],
    #'transform_input': np.array([False, True]),
    # plus parameters for steps:
    # 'scaler__with_mean': [True, False],
    # 'classifier__n_estimators': [100, 200, 300]
}

estimators=[
    ('rf', Pipeline([('rf', model_experiment_3.forest_model.model)])),
    ('gb', Pipeline([('gb', model_experiment_3.gradient_model.model)])),
    ('xgb', Pipeline([('xgb', model_experiment_3.xgboost_model.model)]))
]


grid_search_tomodachi_experiment_3 = GridSearchCV(
    #estimator=model_experiment_3.voting_regressor,
    param_grid=param_grid_experiment_3_forest,
    scoring='r2',
    cv=3,
    verbose=1,
    n_jobs=-1,
    estimator=estimators[0][1]
)


grid_search_tomodachi_experiment_3.fit(X_experiment_3_train, y_experiment_3_train)

best_model_tomodachi_experiment_3 = grid_search_tomodachi_experiment_3.best_estimator_
print("Best parameters:", grid_search_tomodachi_experiment_3.best_params_)
print("Best R2 score on train:", grid_search_tomodachi_experiment_3.best_score_)

"""
For xgboost

Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 100, 'subsample': 0.8}
Best R2 score on train: 0.8220961420049505

Trying Forest that uses 3 models

Fitting 3 folds for each of 2 candidates, totalling 6 fits
Best parameters: {'memory': None}
Best R2 score on train: 0.8178342102571419
"""

Fitting 3 folds for each of 2 candidates, totalling 6 fits
Best parameters: {'memory': None}
Best R2 score on train: 0.8178342102571419


"\nFor xgboost\n\nFitting 3 folds for each of 108 candidates, totalling 324 fits\nBest parameters: {'colsample_bytree': 1.0, 'learning_rate': 0.05, 'max_depth': 7, 'n_estimators': 100, 'subsample': 0.8}\nBest R2 score on train: 0.8220961420049505\n\nTrying voting regressot that uses 3 models\n"

### Experiment - 4 

Now, we wanna see how our features (SPRINT-3) behave and what R2 we get

In [56]:
df_experiment_4 = df.copy()
df_experiment_4["Power_Output"] = df_experiment_4["Power_Output"].shift(-1)
df_experiment_4["Wind_Speed_12h_avg"] = df_experiment_4["Wind_Speed"].rolling(window=12, min_periods=1).mean()
df_experiment_4["Wind_Gust_Diff"] = df_experiment_4["Wind_Gust"] - df_experiment_4["Wind_Speed"]
df_experiment_4["Radiation_Lag1"] = df_experiment_4["Solar_Radiation"].shift(1)

X_experiment_4 = df_experiment_4.drop(columns=["Power_Output", "Precipitation_Unit", "Timestamp"])
X_experiment_4 = X_experiment_4.dropna()

y_experiment_4 = df_experiment_4["Power_Output"]
y_experiment_4 = y_experiment_4.dropna()

In [57]:
X_experiment_4_train, X_experiment_4_test, y_experiment_4_train, y_experiment_4_test = train_test_split(X_experiment_4, y_experiment_4)

In [58]:
X_experiment_4_train

,Wind_Speed,Wind_Gust,Wind_Direction,Temperature,Humidity,Precipitation,Pressure,Cloud_Cover,Solar_Radiation,Hour_of_Day,Day_of_Week,Month,Wind_Speed_Squared,Wind_Speed_Cubed,Wind_Speed_12h_avg,Wind_Gust_Diff,Radiation_Lag1
26376,2.759795,4.233727,22,-8.268020,87.600376,0.287391,1016.823843,18.707844,728.375458,0,2,1,7.616471,21.019903,14.081942,1.473931,674.674196
16405,1.133688,4.617255,266,11.824945,83.486815,4.262946,981.289857,21.045223,628.249105,13,6,11,1.285248,1.457070,10.606691,3.483567,550.020349
8656,12.260810,28.573979,173,-0.589734,73.125241,2.080142,1045.846298,4.565564,783.909172,16,5,12,600.656286,14721.058534,8.646019,16.313169,453.715467
18942,22.754439,25.048096,165,1.465932,71.130017,8.193844,996.974026,21.074059,186.586492,6,0,2,517.764488,11781.440382,13.494299,2.293657,426.660344
3621,2.403703,2.692125,9,14.320155,54.080374,0.522661,1018.883203,21.797393,921.029572,21,5,5,5.777787,13.888084,13.175674,0.288422,343.899499
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9086,3.980757,8.513586,218,-3.507022,86.269656,7.022529,980.255108,37.931398,924.110452,14,2,1,15.846425,63.080764,11.972304,4.532829,657.141990
1828,14.442460,16.000953,77,5.923912,64.217275,8.509687,998.256274,2.213995,873.976656,4,1,3,208.584641,3012.475257,12.193182,1.558493,845.709045
18999,19.454402,20.638144,215,5.523214,79.645059,7.683602,978.780621,75.740173,444.670072,15,2,3,378.473753,7362.980495,10.721620,1.183742,142.225867
7339,0.410364,0.932320,230,1.380909,63.182367,3.159857,1007.391872,73.224575,772.020374,19,6,11,0.168399,0.069105,14.421418,0.521956,399.035095


In [59]:
# Let us create model
model_experiment_4 = TomodachiModel(X_experiment_4_train, y_experiment_4_train)
model_experiment_4.fit_models()


Fitting was successful!
R2 score: 0.9064
RMSE: 131.0732
Fitting was successful!


In [69]:
# No we gonna run. Wait. Yes. We gonna slap GridSearchCV here
param_grid_experiment_4 = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

param_grid_experiment_4 = GridSearchCV(
    estimator=model_experiment_4.xgboost_model.model,
    param_grid=param_grid_experiment_4,
    scoring='r2',
    cv=3,
    verbose=1,
    n_jobs=-1,
)


param_grid_experiment_4.fit(X_experiment_4_train, y_experiment_4_train)

best_model_tomodachi = param_grid_experiment_4.best_estimator_
print("Best parameters:", param_grid_experiment_4.best_params_)
print("Best R2 score on train:", param_grid_experiment_4.best_score_)

Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Best R2 score on train: 0.9283239912538696


### Experiment - 5

This time I wanna try to call Scale. Just one scale and in Experiment 6 i get another one (also separately)

INSIGHT: Scaling is negligible for tree-based model.

In [65]:
df_experiment_5 = df.copy()
df_experiment_5["Power_Output"] = df_experiment_5["Power_Output"].shift(-1)
df_experiment_5["Wind_Speed_12h_avg"] = df_experiment_5["Wind_Speed"].rolling(window=12, min_periods=1).mean()
df_experiment_5["Wind_Gust_Diff"] = df_experiment_5["Wind_Gust"] - df_experiment_5["Wind_Speed"]
df_experiment_5["Radiation_Lag1"] = df_experiment_5["Solar_Radiation"].shift(1)

X_experiment_5 = df_experiment_5.drop(columns=["Power_Output", "Precipitation_Unit", "Timestamp"])
X_experiment_5 = X_experiment_5.dropna()

y_experiment_5 = df_experiment_5["Power_Output"]
y_experiment_5 = y_experiment_5.dropna()

# Balance X_experiment_5
standard_scaler = StandardScaler() # z = (x - u) / s

X_experiment_5_scaled = standard_scaler.fit_transform(X_experiment_5, y_experiment_5)

X_experiment_5_scaled_train, X_experiment_5_scaled_test, y_experiment_5_train, y_experiment_5_test = train_test_split(X_experiment_5_scaled, y_experiment_5)

In [66]:
X_experiment_5_scaled_train

array([[ 1.13742933,  0.8181567 ,  1.18963068, ..., -0.06851132,
        -0.63684926, -1.41308169],
       [ 0.3238843 , -0.0235051 , -0.17349413, ...,  0.84107103,
        -0.60172991, -0.17836363],
       [-1.39392075, -1.69225486, -1.67101153, ...,  0.6768119 ,
        -0.34985896, -1.36104171],
       ...,
       [ 0.717081  ,  0.94205525,  0.02809475, ..., -0.92596874,
         0.29717557, -1.12254698],
       [ 0.02123673, -0.30924103,  0.84404974, ..., -0.83220854,
        -0.5438011 ,  0.67545995],
       [ 1.12450777,  1.29065659, -0.49987613, ...,  0.17049714,
         0.1600878 , -1.21382901]], shape=(22499, 17))

In [67]:
# Create that model lol
model_experiment_5 = TomodachiModel(X_experiment_5_scaled_train, y_experiment_5_train)
model_experiment_5.fit_models()

Fitting was successful!
R2 score: 0.9149
RMSE: 127.2123
Fitting was successful!


In [68]:
# No we gonna run. Wait. Yes. We gonna slap GridSearchCV here
param_grid_experiment_5 = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

param_grid_experiment_5 = GridSearchCV(
    estimator=model_experiment_5.xgboost_model.model,
    param_grid=param_grid_experiment_5,
    scoring='r2',
    cv=3,
    verbose=1,
    n_jobs=-1,
)


param_grid_experiment_5.fit(X_experiment_5_scaled_train, y_experiment_5_train)

best_model_tomodachi = param_grid_experiment_5.best_estimator_
print("Best parameters:", param_grid_experiment_5.best_params_)
print("Best R2 score on train:", param_grid_experiment_5.best_score_)

Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Best R2 score on train: 0.928035762375666


In [ ]:
"""
Results: 

Fitting 3 folds for each of 96 candidates, totalling 288 fits
Best parameters: {'gb__learning_rate': 0.05, 'gb__n_estimators': 100, 'rf__max_depth': 5, 'rf__n_estimators': 200, 'xgb__learning_rate': 0.05, 'xgb__max_depth': 3}
Best R2 score on train: 0.927994647934609

Took 61 minute 9.6s
"""


param_grid_experiment_5_voting_regressor = {
    'rf__n_estimators': [100, 200],
    'rf__max_depth': [5, 7],
    'gb__learning_rate': [0.01, 0.05, 0.1],
    'gb__n_estimators': [100, 200],
    'xgb__learning_rate': [0.01, 0.05],
    'xgb__max_depth': [3, 5]
}

param_grid_experiment_5_voting_regressor = GridSearchCV(
    estimator=model_experiment_5.voting_regressor,
    param_grid=param_grid_experiment_5_voting_regressor,
    scoring='r2',
    cv=3,
    verbose=1,
    n_jobs=-1,
)

param_grid_experiment_5_voting_regressor.fit(X_experiment_5_scaled_train, y_experiment_5_train)

best_model_tomodachi = param_grid_experiment_5_voting_regressor.best_estimator_
print("Best parameters:", param_grid_experiment_5_voting_regressor.best_params_)
print("Best R2 score on train:", param_grid_experiment_5_voting_regressor.best_score_)

Fitting 3 folds for each of 96 candidates, totalling 288 fits
Best parameters: {'gb__learning_rate': 0.05, 'gb__n_estimators': 100, 'rf__max_depth': 5, 'rf__n_estimators': 200, 'xgb__learning_rate': 0.05, 'xgb__max_depth': 3}
Best R2 score on train: 0.927994647934609


### Experiment - 6

I try to use here MiniMax scale and combine all Xs, ys from Experiment 5. I store the params of the model in comment but in future,

we can use Yaml.

In [76]:
df_experiment_6 = df.copy()
df_experiment_6["Power_Output"] = df_experiment_6["Power_Output"].shift(-1)
df_experiment_6["Wind_Speed_12h_avg"] = df_experiment_6["Wind_Speed"].rolling(window=12, min_periods=1).mean()
df_experiment_6["Wind_Gust_Diff"] = df_experiment_6["Wind_Gust"] - df_experiment_6["Wind_Speed"]
df_experiment_6["Radiation_Lag1"] = df_experiment_6["Solar_Radiation"].shift(1)

X_experiment_6 = df_experiment_6.drop(columns=["Power_Output", "Precipitation_Unit", "Timestamp"])
X_experiment_6 = X_experiment_6.dropna()

y_experiment_6 = df_experiment_6["Power_Output"]
y_experiment_6 = y_experiment_6.dropna()


"""
X_std = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
X_scaled = X_std * (max - min) + min
"""


# Balance X_experiment_5
mini_max_scaler = MinMaxScaler() # z = (x - u) / s

X_experiment_6_scaled = mini_max_scaler.fit_transform(X_experiment_6, y_experiment_6)

X_experiment_6_scaled_train, X_experiment_6_scaled_test, y_experiment_6_train, y_experiment_6_test = train_test_split(X_experiment_6_scaled, y_experiment_6)

In [77]:
model_experiment_6 = TomodachiModel(X_experiment_6_scaled_train, y_experiment_6_train)
model_experiment_6.fit_models()

Fitting was successful!
R2 score: 0.9047
RMSE: 132.4317
Fitting was successful!


In [ ]:
param_grid_experiment_6 = {
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

param_grid_experiment_6 = GridSearchCV(
    estimator=model_experiment_6.xgboost_model.model,
    param_grid=param_grid_experiment_6,
    scoring='r2',
    cv=3,
    verbose=1,
    n_jobs=-1,
)


param_grid_experiment_6.fit(X_experiment_6_scaled_train, y_experiment_6_train)

best_model_tomodachi = param_grid_experiment_6.best_estimator_
print("Best parameters:", param_grid_experiment_6.best_params_)
print("Best R2 score on train:", param_grid_experiment_6.best_score_)

Fitting 3 folds for each of 108 candidates, totalling 324 fits
Best parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Best R2 score on train: 0.9268836972655015


### Experiment - 7

Do I try Combine them both (Scale & Max)? Let us try

In [81]:
df_experiment_7 = df.copy()
df_experiment_7["Power_Output"] = df_experiment_7["Power_Output"].shift(-1)
df_experiment_7["Wind_Speed_12h_avg"] = df_experiment_7["Wind_Speed"].rolling(window=12, min_periods=1).mean()
df_experiment_7["Wind_Gust_Diff"] = df_experiment_7["Wind_Gust"] - df_experiment_7["Wind_Speed"]
df_experiment_7["Radiation_Lag1"] = df_experiment_7["Solar_Radiation"].shift(1)

X_experiment_7 = df_experiment_7.drop(columns=["Power_Output", "Precipitation_Unit", "Timestamp"])
X_experiment_7 = X_experiment_7.dropna()

y_experiment_7 = df_experiment_7["Power_Output"]
y_experiment_7 = y_experiment_7.dropna()

"""
We try to use both scales, so we need Pipeline here. Let us try using 
it without our preprocessor:

- StandardScaler 
- MiniMaxScaler

"""

X_experiment_7_train, X_experiment_7_test, y_experiment_7_train, y_experiment_7_test = train_test_split(
    X_experiment_7, y_experiment_7,
    test_size=0.2,
)

experiment_7_pipeline = Pipeline([
    ('standard_scaler', StandardScaler()),
    ('minmax_scaler', MinMaxScaler()),
    ('xgb', xgb.XGBRegressor(objective='reg:squarederror', random_state=42))
])

param_grid_experiment_7 = {
    'xgb__learning_rate': [0.01, 0.05],
    'xgb__n_estimators': [100, 200],
    'xgb__max_depth': [3, 5]
}

grid_search_experiment_7 = GridSearchCV(
    experiment_7_pipeline,
    param_grid_experiment_7,
    scoring='r2',
    cv=3,
    verbose=3,
    n_jobs=-1
)

grid_search_experiment_7.fit(X_experiment_7_train, y_experiment_7_train)
y_experiment_7_pred = grid_search_experiment_7.predict(X_experiment_7_test)
r2 = r2_score(y_experiment_7_test, y_experiment_7_pred)

# Display results
print(f"Best parameters: {grid_search_experiment_7.best_params_}")
print(f"Best R2 score on train: {grid_search_experiment_7.best_score_:.4f}")
print(f"R2 score on test: {r2:.4f}")

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best parameters: {'xgb__learning_rate': 0.05, 'xgb__max_depth': 3, 'xgb__n_estimators': 100}
Best R2 score on train: 0.9254
R2 score on test: 0.9373


In [82]:
df.columns

Index(['Timestamp', 'Wind_Speed', 'Wind_Gust', 'Wind_Direction', 'Temperature',
       'Humidity', 'Precipitation', 'Pressure', 'Cloud_Cover',
       'Solar_Radiation', 'Hour_of_Day', 'Day_of_Week', 'Month',
       'Wind_Speed_Squared', 'Wind_Speed_Cubed', 'Power_Output',
       'Precipitation_Unit'],
      dtype='object')

### Experiment - 8

Uses OneHotEncoding and Sine function for time 

In [15]:
df_experiment_8 = df.copy()
df_experiment_8["Hour_of_Day_sin"] = np.sin(2 * np.pi * df_experiment_8["Hour_of_Day"] / 24)
df_experiment_8["Hour_of_Day_cos"] = np.cos(2 * np.pi * df_experiment_8["Hour_of_Day"] / 24)

df_experiment_8 = df_experiment_8.drop(columns=["Hour_of_Day"])

df_experiment_8["Power_Output"] = df_experiment_8["Power_Output"].shift(-1)
df_experiment_8["Wind_Speed_12h_avg"] = df_experiment_8["Wind_Speed"].rolling(window=12, min_periods=1).mean()
df_experiment_8["Wind_Gust_Diff"] = df_experiment_8["Wind_Gust"] - df_experiment_8["Wind_Speed"]
df_experiment_8["Radiation_Lag1"] = df_experiment_8["Solar_Radiation"].shift(1)

X_experiment_8 = df_experiment_8.drop(columns=["Power_Output", "Precipitation_Unit", "Timestamp"])
X_experiment_8 = X_experiment_8.dropna()

y_experiment_8 = df_experiment_8["Power_Output"]
y_experiment_8 = y_experiment_8.dropna()

X_experiment_8_train, X_experiment_8_test, y_experiment_8_train, y_experiment_8_test = train_test_split(
    X_experiment_8, y_experiment_8,
    test_size=0.2,
)


# OneHotEncoding features:
cat_features = ["Day_of_Week", "Month"]
num_features = [col for col in X_experiment_8.columns if col not in cat_features]


# Preprocessor
preprocessor_experiment_8 = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), num_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_features)
    ]
)


# Final pipeline
pipeline_experiment_8 = Pipeline(steps=[
    ("preprocessor", preprocessor_experiment_8),
    ("regressor", xgb.XGBRegressor(
        learning_rate=0.05,
        max_depth=3,
        n_estimators=100
    ))
])

In [16]:
param_grid_experiment_8 = {
    'regressor__learning_rate': [0.01, 0.05],
    'regressor__n_estimators': [100, 200],
    'regressor__max_depth': [3, 5]
}

grid_search_experiment_8 = GridSearchCV(
    pipeline_experiment_8,
    param_grid_experiment_8,
    scoring='r2',
    cv=3,
    verbose=3,
    n_jobs=-1
)

grid_search_experiment_8.fit(X_experiment_8_train, y_experiment_8_train)
y_experiment_8_pred = grid_search_experiment_8.predict(X_experiment_8_test)
r2 = r2_score(y_experiment_8_test, y_experiment_8_pred)

# Display results
print(f"Best parameters: {grid_search_experiment_8.best_params_}")
print(f"Best R2 score on train: {grid_search_experiment_8.best_score_:.4f}")
print(f"R2 score on test: {r2:.4f}")

Fitting 3 folds for each of 8 candidates, totalling 24 fits
Best parameters: {'regressor__learning_rate': 0.05, 'regressor__max_depth': 3, 'regressor__n_estimators': 100}
Best R2 score on train: 0.9284
R2 score on test: 0.9247


### Final Experiment

Combines all results

In [ ]:
# Let's summarize the best R2 scores from all experiments (based on printed outputs and comments):

# Experiment 1: No preprocessing
# Best R2 score on train: (see cell 17 output)
# Experiment 3: Semen features
# Best R2 score on train: 0.822 (xgboost, see cell 25 comment)
# Experiment 4: SPRINT-3 features
# Best R2 score on train: (see cell 31 output)
# Experiment 5: StandardScaler
# Best R2 score on train: (see cell 36 output)
# Experiment 5 VotingRegressor: 0.92799 (see cell 37 comment)
# Experiment 6: MinMaxScaler
# Best R2 score on train: (see cell 41 output)
# Experiment 7: StandardScaler + MinMaxScaler pipeline
# Best R2 score on train: (see cell 43 output)
# Experiment 8: OneHotEncoding + Sine/Cosine
# Best R2 score on train: (see cell 47 output)

# Based on the comments and outputs, the highest R2 reported is:
best_r2 = 0.92799  # From Experiment 5 VotingRegressor

print(f"The best R2 score achieved today is: {best_r2:.5f} (Experiment 5 VotingRegressor)")

In [17]:
import joblib

# Get the best model from experiment 8
best_model_experiment_8 = grid_search_experiment_8.best_estimator_

# Define export path
export_path = root_dir / "shared" / "data" / "export" / "best_model_experiment_8.joblib"

# Save the model
joblib.dump(best_model_experiment_8, export_path)
print(f"Model saved to {export_path}")


Model saved to C:\Users\Lenovo\Desktop\python_app\tuuleenergia_tomodachi\shared\data\export\best_model_experiment_8.joblib


In [ ]:
import random
chants = ["Tung tung... sahur!", "Tung... sahuuur!", "TungTungTung... sahur!"]

class TungTungSahur:
    """
    AI-generated creature: TungTungSahur
    Attributes:
        energy (int): Current energy level.
        mood (str): Current mood.
    """

    def __init__(self, energy=100):
        self.energy = energy
        self.mood = "mysterious"

    def sahur(self):
        """Performs the legendary sahur ritual, boosting energy."""
        self.energy += 42
        self.mood = "excited"
        print("TungTungSahur chants:", random.choice(chants))

    def dance(self):
        """Dances energetically, consuming energy."""
        if self.energy > 10:
            self.energy -= 10
            self.mood = "dancing"
            print("TungTungSahur performs a rhythmic dance.")
        else:
            self.mood = "tired"
            print("TungTungSahur is too tired to dance.")

    def status(self):
        """Displays current status."""
        print(f"Energy: {self.energy}, Mood: {self.mood}")

# Example usage:
creature = TungTungSahur()
creature.sahur()
creature.dance()
creature.status()

TungTungSahur chants: TungTungTung... sahur!
TungTungSahur performs a rhythmic dance.
Energy: 132, Mood: dancing
